# Comparison between the recommender systems

This notebook compares different recommender systems in terms of various metrics. We benchmark the
algorithms on the MovieLens 100k dataset and Movielens 1M dataset. 

Copyright 2023 Bernardo C. Rodrigues

See COPYING file for license details

In [ ]:
# Setup notebook
import random
import numpy as np
import evaluation.plot as plot

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules every time a cell is executed
%autoreload 2

# Call the function to customize the default template
plot.customize_default_template()

seed = 0
random.seed(seed)
np.random.seed(seed)

In [ ]:
# Load folds
from dataset.movie_lens import load_ml_100k_folds, resolve_folds

folds = resolve_folds(*load_ml_100k_folds(predefined=True))

In [ ]:
# Threshold that defines wether a rating is considered relevant in the metrics computation.
RELEVANCE_THRESHOLD = 4.0

# Number of top recommendations to be considered in the metrics computation (e.g. precision@k).
NUMBER_OF_TOP_RECOMMENDATIONS = 20

# Number of times the each experiment should be repeated. Keep in mind that an experiment already
# involves a 5 fold cross validation.
REPEAT = 1

In [ ]:
# Define the recommender variations to be used in the experiments.
from surprise.prediction_algorithms import (
    SVD,
    KNNWithMeans,
    CoClustering,
)
from recommenders import grecond_recommender, binaps_based_recommenders, common
import evaluation.threads as threads

parallel_recommender_variations = [
    threads.RecommenderVariation(
        "GreConD+IBCAkNN",
        grecond_recommender.GreConDBiAKNNRecommender(
            grecond_coverage=0.1,
            dataset_binarization_threshold=1.0,
            minimum_bicluster_sparsity=None,
            minimum_bicluster_coverage=None,
            minimum_bicluster_relative_size=None,
            knn_type="item",
            user_binarization_threshold=1.0,
            number_of_top_k_biclusters=100,
            knn_k=50,
        ),
    ),
    threads.RecommenderVariation(
        "IBkNN",
        KNNWithMeans(sim_options={"name": "cosine", "user_based": False}, verbose=False),
    ),
    threads.RecommenderVariation(
        "GreConD+UBCAkNN",
        grecond_recommender.GreConDBiAKNNRecommender(
            grecond_coverage=0.2,
            dataset_binarization_threshold=1.0,
            minimum_bicluster_sparsity=None,
            minimum_bicluster_coverage=None,
            minimum_bicluster_relative_size=None,
            knn_type="user",
            user_binarization_threshold=4.0,
            number_of_top_k_biclusters=None,
            knn_k=50,
        ),
    ),
    threads.RecommenderVariation(
        "UBkNN",
        KNNWithMeans(sim_options={"name": "cosine", "user_based": True}, verbose=False),
    ),
    threads.RecommenderVariation("SVD", SVD(verbose=False)),
    threads.RecommenderVariation("CoClustering", CoClustering(verbose=False)),
    (
        "GreConD+MF+UBkNN",
        grecond_recommender.GreConDKNNRecommender(
            grecond_coverage=1.0,
            dataset_binarization_threshold=1.0,
            knn_k=30,
            knn_distance_strategy=common.get_cosine_similarity_matrix,
        ),
    ),
]

sequential_recommender_variations = [
    threads.RecommenderVariation(
        "BinaPs+IBCAkNN",
        binaps_based_recommenders.BinaPsKNNRecommender(
            epochs=100,
            dataset_binarization_threshold=2.5,
            minimum_bicluster_sparsity=None,
            minimum_bicluster_coverage=None,
            minimum_bicluster_relative_size=None,
            knn_type="item",
            user_binarization_threshold=0.0,
            number_of_top_k_biclusters=5,
            knn_k=10,
        ),
    )
]

In [ ]:
# Run the benchmarks on MovieLens 100k
from evaluation.plot import benchmark
from evaluation.threads import generic_benchmark_thread

results = benchmark(
    folds,
    parallel_recommender_variations,
    sequential_recommender_variations,
    REPEAT,
    RELEVANCE_THRESHOLD,
    NUMBER_OF_TOP_RECOMMENDATIONS,
    generic_benchmark_thread,
)

In [ ]:
# Plot results
from evaluation.plot import plot_metric_box_plot, get_result_table
from evaluation.threads import GENERIC_METRIC_NAMES


for metric_name in GENERIC_METRIC_NAMES:
    plot_metric_box_plot(metric_name, results)
    print(get_result_table(metric_name, results))

In [ ]:
# Benchmark MovieLens 1M
from dataset.movie_lens import load_ml_1m_folds
from surprise.prediction_algorithms import KNNBasic, SVD, KNNWithMeans, KNNBaseline
from recommenders import grecond_recommender, binaps_based_recommenders
import evaluation.threads as threads
from evaluation.plot import benchmark
from evaluation.threads import generic_benchmark_thread
from evaluation.plot import plot_metric_box_plot, get_result_table
from evaluation.threads import GENERIC_METRIC_NAMES

folds = load_ml_1m_folds()

# For MovieLens 1M, the best results were obtained with the following parameter:
GRECOND_COVERAGE = 0.2


parallel_recommender_variations = [
    threads.RecommenderVariation(
        "UBkNN (Surprise)",
        KNNBasic(k=KNN_K, sim_options={"name": "cosine", "user_based": True}, verbose=False),
    ),
    threads.RecommenderVariation(
        "IBkNN (Surprise)",
        KNNBasic(k=KNN_K, sim_options={"name": "cosine", "user_based": False}, verbose=False),
    ),
    threads.RecommenderVariation(
        "UBkNN Means (Surprise)",
        KNNWithMeans(k=KNN_K, sim_options={"name": "cosine", "user_based": True}, verbose=False),
    ),
    threads.RecommenderVariation(
        "IBkNN Means (Surprise)",
        KNNWithMeans(k=KNN_K, sim_options={"name": "cosine", "user_based": False}, verbose=False),
    ),
    threads.RecommenderVariation(
        "UBkNN Baseline (Surprise)",
        KNNBaseline(k=KNN_K, sim_options={"name": "cosine", "user_based": True}, verbose=False),
    ),
    threads.RecommenderVariation(
        "IBkNN Baseline (Surprise)",
        KNNBaseline(k=KNN_K, sim_options={"name": "cosine", "user_based": False}, verbose=False),
    ),
    threads.RecommenderVariation("SVD (Surprise)", SVD(verbose=False)),
    threads.RecommenderVariation("SVD++ (Surprise)", SVDpp(verbose=False)),
    threads.RecommenderVariation("SlopeOne (Surprise)", SlopeOne()),
    threads.RecommenderVariation("NMF (Surprise)", NMF(verbose=False)),
    threads.RecommenderVariation("CoClustering (Surprise)", CoClustering(verbose=False)),
    (
        "GreConD+MF+UBkNN",
        grecond_recommender.GreConDKNNRecommender(
            grecond_coverage=1.0,
            dataset_binarization_threshold=1.0,
            knn_k=30,
            knn_distance_strategy=common.get_cosine_similarity_matrix,
        ),
    ),
    threads.RecommenderVariation(
        "GreConD+IBkNN",
        grecond_recommender.GreConDBiAKNNRecommender(
            grecond_coverage=GRECOND_COVERAGE,
            dataset_binarization_threshold=DATASET_BINARIZATION_THRESHOLD,
            minimum_bicluster_sparsity=MINIMUM_BICLUSTER_SPARSITY,
            minimum_bicluster_coverage=MINIMUM_BICLUSTER_COVERAGE,
            minimum_bicluster_relative_size=MINIMUM_BICLUSTER_RELATIVE_SIZE,
            knn_type="item",
            user_binarization_threshold=USER_BINARIZATION_THRESHOLD,
            number_of_top_k_biclusters=NUMBER_OF_TOP_K_BICLUSTERS,
            knn_k=KNN_K,
        ),
    ),
    threads.RecommenderVariation(
        "GreConD+UBkNN",
        grecond_recommender.GreConDBiAKNNRecommender(
            grecond_coverage=GRECOND_COVERAGE,
            dataset_binarization_threshold=DATASET_BINARIZATION_THRESHOLD,
            minimum_bicluster_sparsity=MINIMUM_BICLUSTER_SPARSITY,
            minimum_bicluster_coverage=MINIMUM_BICLUSTER_COVERAGE,
            minimum_bicluster_relative_size=MINIMUM_BICLUSTER_RELATIVE_SIZE,
            knn_type="user",
            user_binarization_threshold=USER_BINARIZATION_THRESHOLD,
            number_of_top_k_biclusters=NUMBER_OF_TOP_K_BICLUSTERS,
            knn_k=KNN_K,
        ),
    ),
]


sequential_recommender_variations = [
    threads.RecommenderVariation(
        "BinaPs+IBkNN",
        binaps_based_recommenders.BinaPsKNNRecommender(
            epochs=EPOCHS,
            dataset_binarization_threshold=DATASET_BINARIZATION_THRESHOLD,
            minimum_bicluster_sparsity=MINIMUM_BICLUSTER_SPARSITY,
            minimum_bicluster_coverage=MINIMUM_BICLUSTER_COVERAGE,
            minimum_bicluster_relative_size=MINIMUM_BICLUSTER_RELATIVE_SIZE,
            knn_type="item",
            user_binarization_threshold=USER_BINARIZATION_THRESHOLD,
            number_of_top_k_biclusters=NUMBER_OF_TOP_K_BICLUSTERS,
            knn_k=KNN_K,
        ),
    )
]

results = benchmark(
    folds,
    parallel_recommender_variations,
    sequential_recommender_variations,
    REPEAT,
    RELEVANCE_THRESHOLD,
    NUMBER_OF_TOP_RECOMMENDATIONS,
    generic_benchmark_thread,
)

for metric_name in GENERIC_METRIC_NAMES:
    plot_metric_box_plot(metric_name, results)
    print(get_result_table(metric_name, results))